#### Loading Modules

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
EMO_DICT = {"angry":0, "disgust":1, "fear":2, "happy":3, "neutral":4, "sad":5, "surprise":6}

#### Loading the model

In [2]:
emotor = tf.keras.models.load_model("../input/soi-hh-model/emotor.h5")

#### Defining Train and Test directory

In [3]:
train_dir = '../input/fer2013-aug/data_aug/train'
test_dir = '../input/fer2013-aug/data_aug/test'
classes = len(os.listdir('../input/fer2013-aug/data_aug/train/'))

In [4]:
def get_file_count(path):
    noOfFiles = 0
    for base, dirs, files in os.walk(path):
        for Files in files:
            noOfFiles += 1
    return noOfFiles

In [5]:
def predictor_one(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img,(48,48))
    img=np.array(img)
    img=img.reshape(1,48,48,1)/255
    predict_x=emotor.predict(img)
    result=np.argmax(predict_x,axis=1)
    return result[0]

In [9]:
from tqdm import notebook as nb
def predictor(path):
    classes = os.listdir(path)
    y_pred = []
    y = []
    pb = nb.tqdm(total = get_file_count(path))
    for emo in classes:
        emo_path = os.path.join(path, emo)
        for img in os.listdir(emo_path):
            pb.update(n=1)
            img_path = os.path.join(emo_path, img)
            y.append(EMO_DICT[emo])
            y_pred.append(predictor_one(img_path))
    return y, y_pred

In [16]:
y_train, y_train_pred = predictor(train_dir)
y_test, y_test_pred = predictor(test_dir)

In [17]:
from sklearn.metrics import classification_report
import seaborn as sn
import matplotlib.pyplot as plt
def complete_report(y, y_pred, title):
    print("Classificationm Report")
    print(classification_report(y, y_pred))
    print()
    print("Confusion Matrix")
    cm = tf.math.confusion_matrix(labels=y, predictions=y_pred)
    plt.figure(figsize=(10, 7))
    sn.heatmap(cm, annot=True, fmt='d')
    plt.xlabel('Predicted')
    plt.ylabel('Truth')
    plt.title(title+" Data")


In [18]:
complete_report(y_train, y_train_pred, "Training")
complete_report(y_test, y_test_pred, "Test")